In [1]:
'''
1. データセットの読み込みと前処理
'''
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# X_train(ndarray): 訓練データ(50000,32,32,3)
# X_test(ndarray): テストデータ(10000,32,32,3)
# y_train(ndarray): 訓練データの正解ラベル(50000,)
# y_test(ndarray): テストデータの正解ラベル(10000,)
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 訓練用とテスト用の画像データを正規化する
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
x_train, x_test = x_train/255.0, x_test/255.0

# 訓練データとテストデータの正解ラベルを10クラスのOne-Hot表現に変換
# y_train: (50000, 10)
# y_test:  (10000, 10)
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

170498071/170498071 [==============================] - 4s 0us/step


In [2]:
'''
2. モデルの生成
'''
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten # core layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D    # convolution layers
from tensorflow.keras import regularizers, optimizers

# 正則化のハイパーパラメーターを設定
weight_decay = 1e-4

# Sequentialオブジェクトを生成
model = Sequential()

# 第1層: 畳み込み層1: 正則化を行う
# (バッチサイズ,32,3,3) -> (バッチサイズ,32,32,32)
model.add(
    Conv2D(
        filters=32,                    # フィルターの数
        kernel_size=(3,3),             # 3x3のフィルターを使用
        input_shape=x_train[0].shape,  # 入力データの形状
        padding='same',                # ゼロパディングを行う
        kernel_regularizer=regularizers.l2(weight_decay),
        activation='relu'              # 活性化関数はReLU
        ))

# 第2層: 畳み込み層2: 正則化を行う
# (バッチサイズ,32,32,32) ->(バッチサイズ,32,32,32)
model.add(
    Conv2D(filters=32,                 # フィルターの数は32
        kernel_size=(3,3),          # 3×3のフィルターを使用
        padding='same',             # ゼロパディングを行う
        kernel_regularizer=regularizers.l2(weight_decay),
        activation='relu'           # 活性化関数はReLU
        ))

# 第3層: プーリング層1: ウィンドウサイズは2×2
# (バッチサイズ,32,32,32) -> (バッチサイズ,16,16,32)
model.add(MaxPooling2D(pool_size=(2,2)))
# ドロップアウト1：ドロップアウトは20％
model.add(Dropout(0.2))

# 第4層: 畳み込み層3　正則化を行う
# (バッチサイズ,16,16,32) ->(バッチサイズ,16,16,64)
model.add(
    Conv2D(filters=64,              # フィルターの数は64
        kernel_size=(3,3),          # 3×3のフィルターを使用
        padding='same',             # ゼロパディングを行う
        kernel_regularizer=regularizers.l2(weight_decay),
        activation='relu'           # 活性化関数はReLU
        ))

# 第5層: 畳み込み層4: 正則化を行う
# (バッチサイズ,64,16,16) ->(バッチサイズ,64,16,16)
model.add(
    Conv2D(filters=64,              # フィルターの数は64
        kernel_size=(3,3),          # 3×3のフィルターを使用
        padding='same',             # ゼロパディングを行う
        kernel_regularizer=regularizers.l2(weight_decay),
        activation='relu'           # 活性化関数はReLU
        ))

# 第6層: プーリング層2: ウィンドウサイズは2×2
# (バッチサイズ,16,16,64) -> (バッチサイズ,8,8,64)
model.add(MaxPooling2D(pool_size=(2,2)))
# ドロップアウト2：ドロップアウトは30％
model.add(Dropout(0.3))

# 第7層: 畳み込み層5: 正則化を行う
# (バッチサイズ,8,8,64) -> (バッチサイズ,8,8,128)
model.add(
    Conv2D(filters=128,             # フィルターの数は128
        kernel_size=(3,3),          # 3×3のフィルターを使用
        padding='same',             # ゼロパディングを行う
        kernel_regularizer=regularizers.l2(weight_decay),
        activation='relu'           # 活性化関数はReLU
        ))

# 第8層: 畳み込み層6: 正則化を行う
# (バッチサイズ,8,8,128) -> (バッチサイズ,8,8,128)
model.add(
    Conv2D(filters=128,             # フィルターの数は128
        kernel_size=(3,3),          # 3×3のフィルターを使用
        padding='same',             # ゼロパディングを行う
        kernel_regularizer=regularizers.l2(weight_decay),
        activation='relu'           # 活性化関数はReLU
        ))

# 第9層: プーリング層3: ウィンドウサイズは2×2
# (バッチサイズ,8,8,128) -> (バッチサイズ,4,4,128)
model.add(MaxPooling2D(pool_size=(2,2)))
# ドロップアウト3：ドロップアウトは40％
model.add(Dropout(0.4))

# Flatten: 4階テンソルから2階テンソルに変換
# (バッチサイズ,4,4,128) -> (バッチサイズ,2048)
model.add(Flatten())

# 第10層: 全結合層
# (バッチサイズ,2048) -> (バッチサイズ,128)
model.add(Dense(128,                # ニューロン数は128
                activation='relu')) # 活性化関数はReLU
# ドロップアウト4：ドロップアウトは40％
model.add(Dropout(0.4))

# 第11層: 出力層
# (バッチサイズ,128) -> (バッチサイズ,10)
model.add(Dense(10,                    # 出力層のニューロン数は10
                activation='softmax')) # 活性化関数はソフトマックス

# Sequentialオブジェクトのコンパイル
model.compile(
    # クロスエントロピー誤差
    loss='categorical_crossentropy',
    # バックプロパゲーションのアルゴリズムにAdamを使用
    # 学習率はデフォルトの0.001
    optimizer=optimizers.Adam(learning_rate=0.001),
    # 学習評価として正解率を指定
    metrics=['accuracy']
)

# モデルのサマリを出力
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 64)        36928     
                                                        

In [ ]:
'''
3. 学習を行う
'''
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau

# val_accuracyの改善が5エポック見られなかったら、学習率を0.5倍する。
reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy', # 監視対象は検証データの精度
    factor=0.1,             # 学習率を減衰させる割合
    patience=5,             # 監視対象のエポック数
    verbose=1,              # 学習率を下げたときに通知する
    mode='max',             # 最高値を監視する
    min_lr=0.0001           # 学習率の下限
    )

# ミニバッチのサイズ
batch_size = 50

# データジェネレーターを生成
# 訓練データ
train_datagen = ImageDataGenerator(
    featurewise_center=True,# データセット全体の平均値を取得
    featurewise_std_normalization=True, # データを標準化する
    width_shift_range=0.1,  # 横サイズの0.1の割合でランダムに水平移動
    height_shift_range=0.1, # 縦サイズの0.1の割合でランダムに垂直移動
    rotation_range=10,      # 10度の範囲でランダムに回転させる
    zoom_range=0.1,         # ランダムに拡大
    horizontal_flip=True)   # 左右反転

# テストデータ
test_datagen = ImageDataGenerator(
    featurewise_center=True,            # データセット全体の平均値を取得
    featurewise_std_normalization=True, # データを標準化する
    )

# ジェネレーターで正規化を行う場合はfit()でデータに適合させる
# 訓練データ
train_datagen.fit(x_train)
# テストデータ
test_datagen.fit(x_test)

# ジェネレーターにミニバッチを適合させる
# 訓練データ
train_generator = train_datagen.flow(
    x_train, # 訓練データ
    y_train, # 正解ラベル
    batch_size=batch_size
    )
# 検証データ
validation_generator = test_datagen.flow(
    x_test, # テストデータ
    y_test, # 正解ラベル
    batch_size=batch_size
    )

# 学習回数
epochs = 60
# 学習を行う
history = model.fit(
    train_generator, # 訓練データ
    epochs=epochs,   # 学習回数
    verbose=1,       # 進捗状況を出力する
    validation_data=validation_generator, # 検証データ
    callbacks=[reduce_lr] # 学習率減衰をコールバック
)

Epoch 1/60
1000/1000 [==============================] - 533s 530ms/step - loss: 1.7093 - accuracy: 0.3754 - val_loss: 1.3264 - val_accuracy: 0.5297 - lr: 0.0010
Epoch 2/60
1000/1000 [==============================] - 521s 521ms/step - loss: 1.3690 - accuracy: 0.5192 - val_loss: 1.1353 - val_accuracy: 0.6149 - lr: 0.0010
Epoch 3/60
1000/1000 [==============================] - 489s 489ms/step - loss: 1.2232 - accuracy: 0.5816 - val_loss: 1.0149 - val_accuracy: 0.6603 - lr: 0.0010
Epoch 4/60
1000/1000 [==============================] - 509s 509ms/step - loss: 1.1213 - accuracy: 0.6247 - val_loss: 0.9866 - val_accuracy: 0.6779 - lr: 0.0010
Epoch 5/60
1000/1000 [==============================] - 512s 512ms/step - loss: 1.0717 - accuracy: 0.6511 - val_loss: 0.9572 - val_accuracy: 0.6871 - lr: 0.0010
Epoch 6/60
1000/1000 [==============================] - 491s 491ms/step - loss: 1.0243 - accuracy: 0.6700 - val_loss: 0.8592 - val_accuracy: 0.7333 - lr: 0.0010
Epoch 7/60
1000/1000 [============

In [ ]:
'''
4. 損失と精度の推移をグラフにする
'''
import matplotlib.pyplot as plt

# 学習結果（損失）のグラフを描画
plt.plot(
    history.history['loss'],
    marker='.',
    label='loss (Training)')
plt.plot(
    history.history['val_loss'],
    marker='.',
    label='loss (Validation)')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

# 学習結果（精度）のグラフを描画
plt.plot(
    history.history['accuracy'],
    marker='.',
    label='accuracy (Training)')
plt.plot(
    history.history['val_accuracy'],
    marker='.',
    label='accuracy (Validation)')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()